In [26]:
%reset

from os import path
import cv2
import os
import random
import numpy as np
from keras.models import Sequential
from keras.layers import Conv3D
from keras.layers.core import Dense,Dropout,Activation
from keras.optimizers import adam
from keras.utils import np_utils
from keras import backend as K


original_data_path='D:\dataset\Image\General-100'



original_data_path

def load_small_img_dataset(path=os.getcwd()):
    data=[]
    for image in os.listdir(path):
        if image.endswith(('.jpg','.jpeg','.png','bmp'),0,len(image)):
            pixels=cv2.imread(os.path.join(path,image))
            pixels=pixels[0:100,0:100]  #first make all images of same size using crop
            data.append(pixels)
            
    return data
    
imgs=load_small_img_dataset(original_data_path)
            
# OR import keras dataset       from keras.datasets import cifar10  
#(x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')





#generate a random number between 0 and 1 and if it is less than 0.7 than the current image is in train set else fi >0.9 then test else dev set

#or use sklearn.model_selection import train_test_split

def test_train_dev_split(dataset,train=0.7,dev=0.2,test=0.1):
    #make seed for exact results everything
    #random.sort(dataset)
    random.seed(2)
    random.shuffle(dataset)
    split1=int(train*len(dataset))
    split2=int((train+dev)*len(dataset))
    
    train_set=dataset[:split1]
    dev_set=dataset[split1:split2]
    test_set=dataset[split2:]
    
    return train_set,dev_set,test_set
    
    
def create_input_out_sets(train_output,dev_output,test_output):
    train_input=[]
    dev_input=[]
    test_input=[]
    for img in train_output:
        low_pixels=cv2.resize(img,None,fx=0.5,fy=0.5,interpolation=cv2.INTER_AREA)
        train_input.append(cv2.resize(low_pixels,None,fx=2,fy=2,interpolation=cv2.INTER_AREA))
    
    for img in dev_output:
        low_pixels=cv2.resize(img,None,fx=0.5,fy=0.5,interpolation=cv2.INTER_AREA)
        dev_input.append(cv2.resize(low_pixels,None,fx=2,fy=2,interpolation=cv2.INTER_AREA))
    
    for img in test_output:
        low_pixels=cv2.resize(img,None,fx=0.5,fy=0.5,interpolation=cv2.INTER_AREA)
        test_input.append(cv2.resize(low_pixels,None,fx=2,fy=2,interpolation=cv2.INTER_AREA))

        
    return np.array(train_input),np.array(train_output),np.array(dev_input),np.array(dev_output),np.array(test_input),np.array(test_output)

def psnr(y_true, y_pred):
    '''assert y_true.shape == y_pred.shape, "Cannot calculate PSNR. Input shapes not same." \
                                         " y_true shape = %s, y_pred shape = %s" % (str(y_true.shape),
                                                                                   str(y_pred.shape))
    '''
    """
    PSNR is Peek Signal to Noise Ratio, which is similar to mean squared error.
    It can be calculated as
    PSNR = 20 * log10(MAXp) - 10 * log10(MSE)
    When providing an unscaled input, MAXp = 255. Therefore 20 * log10(255)== 48.1308036087.
    However, since we are scaling our input, MAXp = 1. Therefore 20 * log10(1) = 0.
    Thus we remove that component completely and only compute the remaining MSE component.
    """
    return -10. * K.log(K.mean(K.square(y_pred - y_true))) / K.log(10.)

# z-score normalization???????

#check wheather the imgs were correctly formed
train,dev,test=test_train_dev_split(imgs)
print("[INFO] Splitting done....")

train_in,train_out,dev_in,dev_out,test_in,test_out=create_input_out_sets(train,dev,test)
print("[INFO] Input and output images created and sorted in datasets")

cv2.imshow("Original Img",test_out[1])
cv2.imshow("Low resolution",test_in[1])
cv2.waitKey(0)
cv2.destroyAllWindows()

print("[INFO] Shape of image",test_in[1].shape)

test_imgs=test_in
test_original_resolution=test_out
#Flatten the images as the model doesnot use 2D structure

train_in=train_in.flatten().reshape(len(train_in),30000)
train_out=train_out.flatten().reshape(len(train_out),30000)
dev_in=dev_in.flatten().reshape(len(dev_in),30000)
dev_out=dev_out.flatten().reshape(len(dev_out),30000)
test_in=test_in.flatten().reshape(len(test_in),30000)
test_out=test_out.flatten().reshape(len(test_out),30000)
print("[INFO] Images flattened")


# z-score normalization???????

# Define the keras DNN model
model =Sequential()
model.add(Dense(1024,input_shape=(30000,)))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(30000))

#compile the model algong with adam optimiser along with PSNR/SSIM loss metric
model.compile(optimizer=adam(0.01),metrics=[psnr],loss='mse')
model.fit(train_in,train_out,batch_size=5,nb_epoch=2,validation_data=(dev_in,dev_out))

#evaluate the model
score=model.evaluate(test_in,test_out)
print("[INFO] MSE:{0}   PSNRLoss:{1}".format(score[0],score[1]))

test_in[0].shape

# prediction

#unknown test data
#cv2.imshow("Original Img",test_original_resolution[0])
#cv2.imshow("Low resolution",test_imgs[0])
cv2.imshow("Peredicted resolution",model.predict(test_in[0:1]).reshape(100,100,3))
cv2.waitKey(0)
cv2.destroyAllWindows()


'''

#Save the model
model.save('resoluteitmodel.h5')
jsonmodel=model.to_json()
model.save_weights('savedWeights.h5')


model.summary()

#loading saved weights
modelWts=model.load_weights('savedWeights.h5')
model.get_weights()
#model.get_config()
'''



'''
PROBLEM:: Cannot restore the image from flattened image, thus we have to use CNN only'''

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
[INFO] Splitting done....
[INFO] Input and output images created and sorted in datasets
[INFO] Shape of image (100, 100, 3)
[INFO] Images flattened


c:\users\ravi\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:140: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 70 samples, validate on 19 samples
Epoch 1/2
70/70 [==============================] - 74s 1s/step - loss: 261488671.3020 - psnr: -78.2477 - val_loss: 16933092.3158 - val_psnr: -72.2170
Epoch 2/2
11/11 [==============================] - 0s 15ms/step
[INFO] MSE:758568.9375   PSNRLoss:-58.7999382019043


"\n\n#Save the model\nmodel.save('resoluteitmodel.h5')\njsonmodel=model.to_json()\nmodel.save_weights('savedWeights.h5')\n\n\nmodel.summary()\n\n#loading saved weights\nmodelWts=model.load_weights('savedWeights.h5')\nmodel.get_weights()\n#model.get_config()\n"